In [ ]:
library(Signac)
library(Seurat)
library(SeuratWrappers)
library(ggplot2)
library(tidyverse)
library(harmony)
library(future)
library(patchwork)
library(EnsDb.Hsapiens.v86)
library(futile.logger)

In [ ]:
setwd("...")

# Load and check

In [ ]:
# rna
obj_rna <- readRDS(file = ".../seu.combined.rds")

In [ ]:
obj_atac <- readRDS(".../combined_filter_final.rds")

In [80]:
Idents(obj_rna) <- "treat"
obj_rna <- RenameIdents(obj_rna,'DMSO' = "dmso",
                        'Inh21' = "inh21")
obj_rna$treat <- Idents(obj_rna)

In [91]:
# first add dataset-identifying metadata
obj_rna$dataset <- "RNAdataset"
obj_atac$dataset <- "ATACdataset"

# handle rna data RNA assay

In [ ]:
DefaultAssay(obj_rna) <- "RNA"
obj_rna <- NormalizeData(obj_rna)
obj_rna <- FindVariableFeatures(obj_rna)
obj_rna <- ScaleData(obj_rna)
obj_rna <- RunPCA(obj_rna, npcs = 50)
PCs <- 30
obj_rna <- RunUMAP(obj_rna, reduction = "pca", dims = 1:PCs) %>%
  FindNeighbors(reduction = "pca", dims = 1:PCs) %>%
  FindClusters(resolution = 0.1, verbose = FALSE) 

# Merge

In [ ]:
common_genes <- intersect(rownames(obj_rna@assays$RNA@counts), rownames(obj_atac@assays$RNA@counts))
combined <- merge(obj_rna, obj_atac, add.cell.ids = c("rna", "atac"))
combined[["RNA"]] <- split(combined[["RNA"]], f = combined$dataset)
combined <- NormalizeData(combined)
combined <- FindVariableFeatures(combined)
combined <- ScaleData(combined)
combined <- RunPCA(combined)

In [ ]:
combined <- FindNeighbors(combined, dims = 1:30, reduction = "pca")
combined <- FindClusters(combined, resolution = 0.1, cluster.name = "unintegrated_clusters")
combined <- RunUMAP(combined, dims = 1:30, reduction = "pca", reduction.name = "umap.unintegrated")

Computing nearest neighbor graph

Computing SNN



# Integration_cca

In [ ]:
combined <- IntegrateLayers(object = combined, method = CCAIntegration, orig.reduction = "pca", new.reduction = "integrated.cca",
    verbose = FALSE)
# re-join layers after integration
combined[["RNA"]] <- JoinLayers(combined[["RNA"]])
inte_pc <- 20
combined <- RunUMAP(combined, dims = 1:inte_pc, reduction = "integrated.cca",reduction.name = "umap.integrated")

In [ ]:
rsl <- seq(0.1, 1, by = 0.1)
combined <- FindNeighbors(combined, reduction = "integrated.cca", dims = 1:inte_pc)
combined <- FindClusters(combined, resolution = rsl)

In [ ]:
Idents(combined) <- "RNA_snn_res.0.5"
all_marker <- FindAllMarkers(combined,assay = "RNA", slot = "data")
clusters_related_markers <- all_marker %>% 
  filter(p_val_adj < 0.05, avg_log2FC > 0, gene %in% gene_list)